In [6]:
import pyodbc
import pandas as pd
from jinja2 import Environment, FileSystemLoader
import json

# Database connection parameters
# Connection parameters
driver = "SQL Server"
server = 'BHTS-CONYPROD16'
database = 'CB_FDM_OBJECTS'
trusted_connection = 'yes'

# Connection string
connection_string = f"""
DRIVER={{{driver}}};
SERVER={server};
DATABASE={database};
Trusted_Connection={trusted_connection};
"""

conn = "";
# Establish the connection
try:
    conn = pyodbc.connect(connection_string)
    print("Connection established successfully.")
except Exception as e:
    print("Error connecting to the database:", e)
  



Connection established successfully.


In [7]:
# Query to fetch metadata
metadata_query = '''
SELECT 
    m.MetadataID, 
    m.DatasetName, 
    m.Description AS DatasetDescription, 
    m.BuildDate, 
    m.DataUpTo, 
    m.ObservationPeriodStart, 
    m.ObservationPeriodEnd, 
    m.AdditionalInfo, 
    t.TableName, 
    t.TableType, 
    t.Description AS TableDescription 
FROM 
    FDM_Metadata m
JOIN 
    FDM_Tables t ON m.MetadataID = t.MetadataID
ORDER BY 
    m.MetadataID, t.TableName;
'''

# Fetch data
metadata_df = pd.read_sql(metadata_query, conn)

# Disconnect from the database
conn.close()



C:\Users\ThirunarayananR\AppData\Local\Temp\ipykernel_2768\2332691216.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  metadata_df = pd.read_sql(metadata_query, conn)


In [8]:
# Organize data into nested structure
metadata_dict = {}
for _, row in metadata_df.iterrows():
    metadata_id = row['MetadataID']
    if metadata_id not in metadata_dict:
        metadata_dict[metadata_id] = {
            'DatasetName': row['DatasetName'],
            'DatasetDescription': row['DatasetDescription'],
            'BuildDate': row['BuildDate'],
            'DataUpTo': row['DataUpTo'],
            'ObservationPeriodStart': row['ObservationPeriodStart'],
            'ObservationPeriodEnd': row['ObservationPeriodEnd'],
            'AdditionalInfo': row['AdditionalInfo'],
            'Tables': []
        }
    metadata_dict[metadata_id]['Tables'].append({
        'TableName': row['TableName'],
        'TableType': row['TableType'],
        'TableDescription': row['TableDescription']
    })

# Convert the dictionary to a list
metadata_list = list(metadata_dict.values())



In [9]:
# Save the metadata records as a JSON file
with open('metadata.json', 'w') as json_file:
    json.dump(metadata_list, json_file, indent=4)

# Setup Jinja2 environment
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('template.html')

# Render the HTML content
html_content = template.render(metadata=metadata_list)

# Write to an HTML file
with open('metadata.html', 'w') as f:
    f.write(html_content)

print("HTML file 'metadata.html' has been created.")

HTML file 'metadata.html' has been created.
